modif 1

In [60]:
# Importation des librairies
import pandas as pd
# Importation de la classe de forêt aléatoire
from sklearn.ensemble import RandomForestClassifier

# Création d'une classe de forêt aléatoire avec seuil de probabilité
# basée sur la classe Random de sklearn.linear_model
# pour détecter les billets supérieurs à une petite probabilité d'être faux
# On choisira ici 16%
class RandomForestClassifier_seuil(RandomForestClassifier):
    def __init__(self, seuil = 0.5, **kwargs):
        kwargs.pop(seuil, None)
        super().__init__(**kwargs)
        self.seuil = seuil
    def predict(self, X_loc):
            y_hat_proba_loc = self.predict_proba(X_loc)[:,1]
            return [0 if value < self.seuil else 1 for value in y_hat_proba_loc]

In [61]:
# Importation des données d'entraînement et prépation
billets_full =pd.read_csv("ressources/billets_full.csv", sep=";")
dimensions = ['height_left', 'height_right', 'margin_low', 'margin_up', 'length']
X_train = billets_full[dimensions]
y_train = billets_full["is_fake"]

In [62]:
# Création du modèle avec probabilité fixée à 16 %
rf_seuil = RandomForestClassifier_seuil(seuil = 0.16, max_depth = 6)

# Entraînement sur le jeu préparé
rf_seuil.fit(X_train, y_train)

RandomForestClassifier_seuil(seuil=0.16)

In [66]:
#Importation du jeu de production
production =pd.read_csv("ressources/billets_production.csv", sep=",") 

# Définition du jeu d'entrée du modèle
X_production = production[dimensions]

#Prédiction sur le jeu de production
y_production = rf_seuil.predict(X_production)

# Conversion de y_production (=is_fake) en is_genuine
production['is_genuine'] = pd.Series(y_production).replace({0:True,1:False})

#Prédiciton des probabilités des billets d'être vrais
production['is_genuine_proba'] = 1-pd.Series(rf_seuil.predict_proba(X_production)[:,1])

# Export en csv
production.to_csv("ressources/resultat_production.csv", sep=";", index = False)

production.style.format({'is_genuine_proba':'{0:.6f}'}, precision = 2)

C:\Users\DELL\AppData\Local\Temp\ipykernel_684\347938638.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  production['is_genuine'] = pd.Series(y_production).replace({0:True,1:False})


,diagonal,height_left,height_right,margin_low,margin_up,length,id,is_genuine,is_genuine_proba
0,172.09,103.95,103.73,4.39,3.09,113.19,B_1,True,0.999105
1,171.52,104.17,104.03,5.27,3.16,111.82,B_2,False,0.000333
2,171.78,103.80,103.75,3.81,3.24,113.39,B_3,True,0.996436
3,172.02,104.08,103.99,5.57,3.30,111.10,B_4,False,0.000414
4,171.79,104.34,104.37,5.00,3.07,111.87,B_5,False,0.000231


Prédictions de probabilités que chaque billet soit faux : 